In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18629161
paper_name = 'tucker_fields_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/SUMMARY6-22.xlsx', sheet_name='orf_geneontology.tab')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4542 x 15


In [7]:
original_data['orf'] = original_data['ORF'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [9]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene, per, oldper, men, oldmen, mef, oldmef, ibu, oldibu, process, function, localization, plate, spot, orf]
Index: []


In [11]:
original_data.set_index('orf', inplace=True)

In [12]:
original_data = original_data[['per','men','mef','ibu']]

In [16]:
for c in original_data.columns:
    original_data[c] = pd.to_numeric(original_data[c], errors='coerce')

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data = 1 / original_data

In [21]:
original_data.shape

(4535, 4)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [16259,16285,16287,16286]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,16259,16285,16287,16286
data_type,value,value,value,value
orf,,,,
YAL002W,0.701754,1.030928,1.084270,0.769231
YAL004W,0.826446,0.917431,1.102681,1.052632
YAL005C,0.943396,1.098901,1.117277,0.980392
YAL007C,0.840336,0.961538,0.992465,0.952381
YAL008W,0.980392,1.063830,1.019341,1.041667


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 12


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [28]:
data.head()

,dataset_id,16259,16285,16287,16286
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,0.701754,1.030928,1.084270,0.769231
1863,YAL004W,0.826446,0.917431,1.102681,1.052632
4,YAL005C,0.943396,1.098901,1.117277,0.980392
5,YAL007C,0.840336,0.961538,0.992465,0.952381
6,YAL008W,0.980392,1.063830,1.019341,1.041667


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan

In [32]:
data_all = data.join(data_norm)

In [33]:
data_all.head()


,dataset_id,16259,16285,16287,16286,16259,16285,16287,16286
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,0.701754,1.030928,1.084270,0.769231,-0.761770,0.176786,0.586266,-1.504296
1863,YAL004W,0.826446,0.917431,1.102681,1.052632,-0.399521,-0.525447,0.716058,0.391081
4,YAL005C,0.943396,1.098901,1.117277,0.980392,-0.059763,0.597353,0.818960,-0.092054
5,YAL007C,0.840336,0.961538,0.992465,0.952381,-0.359169,-0.252544,-0.060930,-0.279393
6,YAL008W,0.980392,1.063830,1.019341,1.041667,0.047716,0.380358,0.128540,0.317748


# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [37]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 18629161...
Inserting the new data...


100%|██████████| 4/4 [00:25<00:00,  6.30s/it]

Updating the data_modified_on field...
